# Progetto Finale di introduzione alla DataScience
Con questo notebook ci poniamo l'obiettivo di:
1. capire quale sia la categoria di applicazioni avente il massimo rapporto tra numero di recensioni e numero di download (che, da adesso, per brevità, chiameremo nr/nd)

2. capire, dentro a questa categoria, se sono generalmente le app a pagamento o quelle free ad avere un nr/nd più alto

3. investigare l'esistenza di una eventuale correlazione tra il nr/nd e il rating di una applicazione: se questa metrica cresce, generalmente, il rating aumenterà? Diminuirà? Nessuna delle due? Proveremo a scoprirlo

Le ipotesi formulate sono quindi le seguenti:
1. immaginiamo che la categoria di applicazioni avente il massimo rapporto tra numero di recensioni e numero di download siano le app di categoria PRODUCTIVITY (e pensiamo che la categoria SOCIAL sia la seconda) 
2. immaginiamo che, all'interno della categoria trovata, generalmente, le app a pagamento avranno un nr/nd più alto rispetto a quelle free (perché pensiamo che un utente sia più motivato a lasciare una recensione, sia esponendo criticità che punti di forza, quando ha pagato per un prodotto, piuttosto che quando ne ha usufruito gratis)
3. immaginiamo che, al crescere del rapporto nr/nd diminuisca il rating (perché pensiamo che, di solito, se la maggior parte degli utilizzatori lascia una recensione, è per criticare qualcosa piuttosto che per esprimere soddisfazione)




 <br>Trovare la categoria di app che "eccelle" più delle altre in questa metrica potrebbe aiutare, ad esempio, una software house che vuole sviluppare applicazioni particolarmente innovative o che per qualche motivo hanno bisogno di tanti feedback anche dopo varie release: conoscendo i risultati di questa analisi, saprà su quale categoria puntare, avendo trovato la user-base più "attiva" possibile.

## Pulizia ed Analisi Iniziale


### Diamo una occhiata al dataset

In [ ]:
## importiamo il necessario:
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import numpy as np
import scipy.stats as stats
import sklearn
#importiamo il dataset e visualizziamo prime 5 righe:
playstore = pd.read_csv('googleplaystore.csv')
playstore.head()

In [ ]:
#visualizziamo numero di righe e colonne:
print("Numero Righe, Numero Colonne",playstore.shape)

In [ ]:
#visualizziamo le informazioni del dataset:
playstore.info()

In [ ]:
#visualizziamo eventuali valori nulli:
playstore.isnull().sum()

Come vediamo ci sono ben 1474 righe che hanno valore nullo sulla colonna Rating, potremmo pensare di
* rimuovere tutte queste righe (e quindi ridurre significativamente la dimensione del nostro dataset)
* "completare" i valori nulli usando il valore medio del rating.

Optiamo per la prima opzione, in quanto la seconda potrebbe portare ad una distorsione dei dati per quanto riguarda l'analisi che ci proponiamo di attuare, infatti, se pensiamo di voler cercare una correlazione tra nr/nd e Rating ed usiamo il valore medio in tutte le righe che presentano attualmente un valore nullo, avremo dei risultati irrealistici in tutte queste righe,

#### *esempio*:
il rating medio è 2.3, quindi in 1474 entrate avremo il valore 2.3 sulla colonna Rating, ciò significa che poi, quando andremo a cercare una correlazione, potremmo avere:
* App 1: rating = 2.3, nr/nd = 40%
* App 2: rating = 2.3, nr/nd = 60%
* App 3: rating = 2.3, nr/nd = 99%

e la nostra analisi è stata inquinata, quando magari, rimuovendo le entrate in cui Rating è nullo avremmo trovato una correlazione

### Puliamo il dataset

In [ ]:
#eliminiamo le righe con valori nulli nella colonna 'Rating':
playstore = playstore.dropna(axis=0, subset=['Rating'])
playstore.isnull().sum()

Analizzando il dataset individuiamo le caratteristiche che, a nostro parere, dovrebbero essere quantitative:
* Rating 
* Reviews
* Installs
* Size
* Price


In [ ]:
playstore.describe() #visualizziamo statistiche del dataset

Notiamo che utilizzando il metodo describe abbiamo informazioni solo sulla colonna Rating, come mai le altre (anche se pensiamo essere quantitative) non vengono mostrate?<br>Perché contengono valori come stringhe.

#### Pulizia righe rumorose
Inoltre notiamo qualcosa di "curioso" rispetto al valore massimo della colonna Rating, se i rating sul playstore vanno da 1 a 5 stelle, come è possibile che il valore massimo sia 19? C'è una riga problematica nel nostro dataset, che decidiamo di eliminare, in quanto, probabilmente a causa di un errore nel processo di raccolta dei dati, risulta "disallineata", ovvero i valori sono spostati di una colonna:

In [ ]:
righe_rumorose = playstore[(playstore['Rating']>5) | (playstore['Rating']<1)]
print("Numero di righe problematiche rispetto al rating: ",righe_rumorose.shape[0]) # controlliamo quante sono, e scopriamo che c'è una sola riga di questo tipo
(righe_rumorose.head(1)) # quindi la visualizziamo con head

I valori risultano chiaramente spostati di una colonna (dovrebbe essere Rating = 1.9, Reviews=19, Size = 3.0M, eccetera), ma se volessimo "sistemare" questa colonna ci troveremmo con un valore nullo dentro alla colonna Category, il che non ci fa comodo perché vogliamo fare l'analisi proprio su questa colonna. Quindi decidiamo di eliminare interamente la riga.

In [ ]:
(playstore.describe()) # prima di eliminarla, ri-visualizziamo le statistiche del dataset


In [ ]:
playstore = playstore.drop(righe_rumorose.index, axis=0) # eliminiamo la riga rumorosa
playstore.describe() # visualizziamo le statistiche del dataset dopo aver eliminato la riga rumorosa

#### Pulizia dei valori duplicati

In [ ]:
#visualizziamo il numero di valori duplicati quindi con lo stesso valore sotto la colonna app:
print("Numero di valori duplicati: ",playstore.duplicated(subset=['App']).sum())
#eliminiamo i valori duplicati:
playstore = playstore.drop_duplicates(subset=['App'])
#verifichiamo che non ci siano più valori duplicati:
print("Numero di valori duplicati: ",playstore.duplicated(subset=['App']).sum())


Torniamo sulle variabili Install, Reviews, Price e Size, che dovrebbero risultare quantitative ma non accade. Prima di tutto guardiamo il tipo

In [ ]:
print("Tipo di Installs:\t" + str(type(playstore['Installs'][0])))
print("Tipo di Reviews:\t" + str(type(playstore['Reviews'][0])))
print("Tipo di Price:\t\t" + str(type(playstore['Price'][0])))
print("Tipo di Size:\t\t" + str(type(playstore['Size'][0])))

Notiamo che sono appunto tutte stringhe.
Le uniche colonne che però ci interessa realmente pulire e "rendere quantitative" sono: Installs e Reviews, in quanto Price e Size non giocano alcun ruolo nella nostra analisi.

#### Trasformiamo Installs
Installs contiene valore in formato stringa perché il numero di download è espresso nella forma n+, per indicare che l'app è stata scaricata "più di n volte"

In [ ]:
if(type(playstore['Installs'][0]) is str):
    playstore['Installs'] = [int(x.replace('+','').replace(',','')) for x in playstore['Installs']]
# ri-controlliamo le statistiche:
playstore.describe()

#### Trasformiamo Reviews
Reviews contiene valore in formato stringa, lo trasformiamo in numero

In [ ]:
if(type(playstore['Reviews'][0]) is str):
    playstore['Reviews'] = playstore['Reviews'].astype(int)
# ri-controlliamo le statistiche:
playstore.describe()

#### Drop di colonne che non utilizzeremo
Per completare la pulizia, infine, eliminiamo dal dataset le colonne Price e Size (menzionate sopra) e Content Rating, Genres, Last Updated, Current Ver e Android Ver perché di nulla utilità rispetto al nostro obiettivo

In [ ]:
playstore.drop(columns=['Price', 'Size', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'], inplace=True)
playstore.info()


#### Considerazione sul numero di download:
Ha senso, ai fini della nostra analisi, considerare applicazioni con un numero relativamente basso di installazioni?<br>
Immaginiamo lo sviluppatore 'indie' che fa scaricare e recensire la sua app appena sviluppata a 5 suoi conoscenti, quando andremo a calcolare il nostro rapporto nr/nd avremo 100%, ma è davvero significativo?<br>
Risulta essere un indicatore realmente affidabile su un così basso numero di utenti?<br>
Per questo motivo decidiamo di non considerare le applicazioni con meno di 10000 Installs

In [ ]:
# togliamo tutte le applicazioni con meno di 10000 installs:
playstore = playstore[playstore['Installs'] >= 10000]

#### Calcolo del rapporto nr/nd:

In [ ]:
# calcoliamo il rapporto tra numero di recensioni e numero di installazioni:
playstore['nr/nd'] = playstore['Reviews'] / playstore['Installs'] * 100
playstore.describe()
# ordiniamo il dataset in base al rapporto tra numero di recensioni e numero di installazioni:
playstore = playstore.sort_values(by='nr/nd', ascending=False)
playstore.head(20) # diamo una occhiata alle prime 20 righe

#### Problema di precisione nei dati
Notiamo valori sospetti dopo aver calcolato nr/nd ed aver ordinato le righe del dataframe in base a questa metrica (ad esempio un rapporto del 102% dovuto al fatto che ci fossero 10249 reviews per una applicazione installata 10000 volte) questo ci ha portato a ulteriori controlli sui dati. <br>
Siamo ritornati a pulire i dati, in quanto non è possibile che, per una certa applicazione, ci siano più reviews che installazioni.
Ma perché accade questo? Con tutta probabilità, ciò è causato dal fatto che il numero di installazioni non è un numero preciso, bensì un range (infatti come abbiamo detto in precedenza, era indicato nella forma del tipo n+), questo significa che, se una applicazione ha "1+ downloads" essa ha un numero di downloads maggiore o uguale a 1 e però minore di 5, se una applicazione ha un numero di downloads listato come 1,000+ significa che ha tra le 1,000 e  le 5,000 installazioni, e così via.
Mentre, il numero di reviews è un numero preciso.
Per cercare di arginare questo problema procediamo eliminando dal dataset le righe in cui il nr/nd è superiore a 100


In [ ]:
# eliminiamo dal dataset le righe in cui nr/nd è maggiore di 100:
playstore = playstore[playstore['nr/nd'] <= 100]
playstore.describe()

&emsp;&emsp;&emsp;&emsp;&emsp;👆🏼Notiamo anche che considerando solo le app con 10,000 download o più, il rating minimo si è alzato da 1 a 1.6

### Distribuzione delle categorie
Una volta completata la pulizia, iniziamo a cercare di capire meglio la struttura dei nostri dati; cominciamo visualizzando la distribuzione delle categorie delle app nello store

In [ ]:
import matplotlib.pyplot as plt

# Genera una lista di 33 colori diversi
color_palette = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                 '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
                 '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5',
                 '#c49c94', '#f7b6d2', '#c7c7c7', '#dbdb8d', '#9edae5',
                 '#393b79', '#ff6f69', '#5d5d5d', '#ffb347', '#84dcc6',
                 '#595959', '#f6b93b', '#bae1ff', '#a05195', '#ff7c43',
                 '#00818a', '#e55e5e', '#3caea3']

percentages = (playstore['Category'].value_counts() / len(playstore)) * 100
labels = [f'{label} - {percentage:.1f}%' for label, percentage in zip(percentages.index, percentages)]

# Plot del grafico a torta
pie = playstore['Category'].value_counts().plot.pie(fontsize=10, autopct='%1.1f%%', figsize=(10,10), colors=color_palette, labels=None, labeldistance=0.7)
plt.ylabel('')
plt.title('Distribuzione delle categorie')
plt.subplots_adjust(right=1.7)  # spazio a destra per la legenda

# Modifica la dimensione del font delle etichette in base alla dimensione delle fette
sizePerc=20
count=1
for text in pie.texts:
    text.set_fontsize(sizePerc)
    if (count<=33):
        print(count)
        print(sizePerc)
        sizePerc-=0.44
    count+=1
count=1
sizePerc=20
plt.legend(labels=labels, loc='upper right', bbox_to_anchor=(1.3, 1))
# Mostra il grafico
plt.show()



## Risoluzione obiettivo 1:
Cerchiamo di capire quale è la categoria con il rapporto tra numero di recensioni e numero di installazioni più alto

In [ ]:
playstore.groupby('Category')['nr/nd'].mean().sort_values(ascending=False).plot.bar(figsize=(10,5)) # ordiniamo le categorie in base al rapporto tra numero di recensioni e numero di installazioni, prendendo ogni categoria e calcolando la media di nr/nd
plt.ylabel('nr/nd')
plt.title('Rapporto tra numero di recensioni e numero di installazioni per categoria')
plt.show()

Abbiamo quindi scoperto che la categoria di applicazioni con nr/nd più alto è la categoria GAME, differentemente da quanto ci aspettavamo (avendo inizialmente ipotizzato PRODUCTIVITY)

## Olap
Prepariamo i dati per la rappresentazione multidimensionale e iniziamo a visualizzare

In [ ]:
# Recuperiamo le diverse categorie per la rappresentazione OLAP
quantize_category = np.unique(playstore['Category'])
print("Numero di categorie: ", len(quantize_category))
quantize_rating = ['BAD', 'SUFFICIENT', 'GOOD', 'EXCELLENT']
quantize_nr_nd = ['LOW', 'MEDIUM', 'HIGH']


playstore.info()



In [ ]:
playstore.loc[playstore['nr/nd'].between(0, 15), 'nr/nd_quant'] = quantize_nr_nd[0]
playstore.loc[playstore['nr/nd'].between(15, 35), 'nr/nd_quant'] = quantize_nr_nd[1]
playstore.loc[playstore['nr/nd'].between(35, 100), 'nr/nd_quant'] = quantize_nr_nd[2]

playstore.loc[playstore['Rating'] < 2.5, 'Rating_quant'] = quantize_rating[0]
playstore.loc[playstore['Rating'].between(2.5, 3.5), 'Rating_quant'] = quantize_rating[1]
playstore.loc[playstore['Rating'].between(3.6, 4.5), 'Rating_quant'] = quantize_rating[2]
playstore.loc[playstore['Rating'] >4.5, 'Rating_quant'] = quantize_rating[3]



In [ ]:
results = pd.DataFrame(columns=['Category', 'Rating', 'nr/nd', 'Count'])
playstore['combined'] = playstore['Category'] + " / " + playstore['Rating_quant'] + " / " + playstore['nr/nd_quant']
playstore.head()


In [ ]:
counts = playstore['combined'].value_counts()

for z in range(0,len(quantize_category)):
    for j in range(0,len(quantize_rating)):
        for i in range(0,len(quantize_nr_nd)):
            key = f"{quantize_category[z]} / {quantize_rating[j]} / {quantize_nr_nd[i]}"
            #print("Key: ", key) # per vedere come si chiama la chiave (debug)
            count = counts.get(key, 0)
#Aggiungiamo i risultati al dataframe
            results.loc[len(results)] = [quantize_category[z], quantize_rating[j], quantize_nr_nd[i], count]
results.head(1000)


Visualizziamo ricordandoci che abbiamo:
* 33 possibili valori per Category
* 4 possibili valori per Rating
* 3 possibili valori per nr/nd

In [ ]:
# printiamo il valore massimo di count
print("Massimo: ", results['Count'].max())
# ordiniamo il dataframe results in base alla colonna count
results = results.sort_values(by='Count', ascending=False)
results.head(10)

### Interpretazione della visualizzazione qua sopra
La tabella results di cui abbiamo visualizzato alcune righe ci comunica le seguenti informazioni riguardo al nostro dataset, rispondendo a questa domanda:<br>
Quante sono le applicazioni della categoria X che abbiano un rating buono (o cattivo) ed un nr/nd basso (o medio, o alto)? <br>La risposta a questa domanda viene fornita dalla colonna Count, che conta per l'appunto il numero di occorrenze di tali applicazioni.
#### *esempio*:
ci sono 901 applicazioni della categoria FAMILY che hanno un rating GOOD ed un nr/nd LOW.

### Prima visualizzazione OLAP proposta
Siccome abbiamo notato che la categoria FAMILY è quella che si "prende la fetta più grande" tra le categorie di applicazioni dello store, e che GAME è quella mediamente con il rapporto nr/nd più alto, pensiamo sia interessante fare una visualizzazione OLAP per quanto riguarda queste categorie.

#### Slicing su app di categoria FAMILY

In [ ]:
family_results = results.loc[results['Category'] == 'FAMILY']
print(family_results)

In [ ]:
# Trasformiamo i dati in una matrice utilizzando il metodo pivot
matrix = family_results.pivot(index='Rating', columns='nr/nd', values='Count')
# Creiamo la heatmap utilizzando imshow
plt.imshow(matrix, cmap='YlGnBu', interpolation='nearest', norm=mcolors.LogNorm(vmin=1, vmax=1000))

# Aggiungiamo una colorbar per indicare il valore dei colori
cbar = plt.colorbar(label='Count')

# Impostiamo le etichette degli assi
plt.xticks(np.arange(len(matrix.columns)), matrix.columns)
plt.yticks(np.arange(len(matrix.index)), matrix.index)

plt.title('HeatMap dello slicing su FAMILY')

# Modifichiamo le etichette della colorbar, ci sembra più chiaro avere 1, 10, 100, 1000 espressi così, piuttosto che come potenze di 10 (di default)
ticks = [1, 10, 100, 1000]
cbar.set_ticks(ticks)
cbar.set_ticklabels(ticks)
# Mostriamo il grafico
plt.show()

Con questo slicing notiamo che la maggior parte delle app per famiglie hanno un nr/nd basso e che la maggior parte di esse hanno un rating buono.
In particolare, abbiamo visto precedentemente che le app di categoria family sono le più numerose, con una percentuale del 18.7 %. Le app di questa categoria sono 1201 e con questa visualizzazione OLAP vediamo come addirittura circa 900 siano quelle con nr/nd basso e valutazione buona, quindi stiamo parlando del 75% delle applicazioni FAMILY.

#### Slicing su app di categoria GAME

In [ ]:
game_results = results.loc[results['Category'] == 'GAME']
print(game_results)

In [ ]:
# Trasformiamo i dati in una matrice utilizzando il metodo pivot
matrix = game_results.pivot(index='Rating', columns='nr/nd', values='Count')
# Creiamo la heatmap utilizzando imshow
plt.imshow(matrix, cmap='YlGnBu', interpolation='nearest', norm=mcolors.LogNorm(vmin=1, vmax=1200))

# Aggiungiamo una colorbar per indicare il valore dei colori
cbar = plt.colorbar(label='Count')

# Impostiamo le etichette degli assi
plt.xticks(np.arange(len(matrix.columns)), matrix.columns)
plt.yticks(np.arange(len(matrix.index)), matrix.index)

plt.title('HeatMap dello slicing su GAME')


# Modifichiamo le etichette della colorbar, ci sembra più chiaro avere 1, 10, 100, 1000 espressi così, piuttosto che come potenze di 10 (di default)
ticks = [1, 10, 100, 1000]
cbar.set_ticks(ticks)
cbar.set_ticklabels(ticks)
# Mostriamo il grafico
plt.show()

Forse stiamo iniziando a notare un trend? Sia in questo slicing su GAME, che in quello su FAMILY, c'è una totale assenza di occorrenze delle combinazioni:
* (nr/nd medio, rating cattivo)
* (nr/nd alto, rating cattivo)
* (nr/nd alto, rating sufficiente)
* (nr/nd medio, rating sufficiente).
<br>

Con il codice qua sotto ci convinciamo che questo accade **su tutto il dataset** e non solo nelle categorie FAMILY e GAME

In [ ]:
# Filtraggio dei dati per le combinazioni specifiche
bad_medium = results.loc[(results['Rating'] == 'BAD') & (results['nr/nd'] == 'MEDIUM')]
bad_high = results.loc[(results['Rating'] == 'BAD') & (results['nr/nd'] == 'HIGH')]
sufficient_high = results.loc[(results['Rating'] == 'SUFFICIENT') & (results['nr/nd'] == 'HIGH')]
sufficient_medium= results.loc[(results['Rating'] == 'SUFFICIENT') & (results['nr/nd'] == 'MEDIUM')]
# Controllo se tutti i valori nella colonna 'Count' sono zero
print("Tutti zeri per combinazioni (nr/nd medio, rating cattivo) di qualsiasi categoria:", bad_medium['Count'].all() == 0)
print("Tutti zeri per combinazioni (nr/nd alto, rating cattivo) di qualsiasi categoria:", bad_high['Count'].all() == 0)
print("Tutti zeri per combinazioni (nr/nd alto, rating sufficiente) di qualsiasi categoria:", sufficient_high['Count'].all() == 0)
print("Tutti zeri per combinazioni (nr/nd medio, rating sufficiente) di qualsiasi categoria:", sufficient_medium['Count'].all() == 0)


#### Dicing
A questo punto effetuiamo un dicing, visualizzando le app che hanno un Rating "SUFFICIENT" ed un nr/nd "LOW", in quanto è la combinazione che sembra differire in maniera più significativa guardando le due matrici di FAMILY e GAME

In [ ]:
suff_low_apps = results.loc[(results['Rating'] == 'SUFFICIENT') & (results['nr/nd'] == 'LOW')]
print(suff_low_apps)


In [ ]:
#rimuoviamo tutte quelle con count = 0
suff_low_apps_filtered = suff_low_apps.loc[suff_low_apps['Count']>0]
print(suff_low_apps_filtered)

In [ ]:
# Creiamo un bar plot con larghezza delle barre regolata
plt.figure(figsize=(28,12))  ## width, height
plt.bar(suff_low_apps_filtered['Category'], suff_low_apps_filtered['Count'], color=['blue', 'orange', 'red', 'yellow', 'green', 'violet', 'black', 'pink'], width=0.8)


# Impostiamo le etichette degli assi
plt.xlabel('Category', fontsize=20)
plt.ylabel('Count', fontsize=20)

# Impostiamo il titolo del grafico
plt.title('Numero delle app con un rating SUFFICIENT ed un nr/nd LOW per categoria', fontsize=16)

# Impostiamo la dimensione del font per le etichette sull'asse x
plt.xticks(fontsize=10, rotation=58)

# Mostriamo il grafico
plt.show()


Facciamo la stessa cosa, visualizzando adesso le app che hanno un Rating "GOOD" ed un nr/nd "MEDIUM"

In [ ]:
good_medium_apps = results.loc[(results['Rating'] == 'GOOD') & (results['nr/nd'] == 'MEDIUM')]
#rimuoviamo tutte quelle con count = 0
good_medium_apps_filtered = good_medium_apps.loc[good_medium_apps['Count']>0]
print(good_medium_apps_filtered)

In [ ]:
# Creiamo un bar plot con larghezza delle barre regolata
plt.figure(figsize=(18,12))  ## width, height
plt.bar(good_medium_apps_filtered['Category'], good_medium_apps_filtered['Count'], color=['blue', 'orange', 'red', 'yellow', 'green', 'violet', 'black', 'pink'], width=0.8)


# Impostiamo le etichette degli assi
plt.xlabel('Category', fontsize=20)
plt.ylabel('Count', fontsize=20)

# Impostiamo il titolo del grafico
plt.title('Numero delle app con un rating GOOD ed un nr/nd MEDIUM per categoria', fontsize=16)

# Impostiamo la dimensione del font per le etichette sull'asse x
plt.xticks(fontsize=10, rotation=45)

# Mostriamo il grafico
plt.show()

### Seconda visualizzazione OLAP proposta
Vogliamo cambiare il punto di vista dell'OLAP e concentrarci sulle differenze tra app free e paid

In [ ]:
quantize_type = np.unique(playstore['Type'])
quantize_rating = ['BAD', 'SUFFICIENT', 'GOOD', 'EXCELLENT']
quantize_nr_nd = ['LOW', 'MEDIUM', 'HIGH']

playstore.loc[playstore['nr/nd'].between(0, 15), 'nr/nd_quant'] = quantize_nr_nd[0]
playstore.loc[playstore['nr/nd'].between(15, 35), 'nr/nd_quant'] = quantize_nr_nd[1]
playstore.loc[playstore['nr/nd'].between(35, 100), 'nr/nd_quant'] = quantize_nr_nd[2]

playstore.loc[playstore['Rating'] < 2.5, 'Rating_quant'] = quantize_rating[0]
playstore.loc[playstore['Rating'].between(2.5, 3.5), 'Rating_quant'] = quantize_rating[1]
playstore.loc[playstore['Rating'].between(3.6, 4.5), 'Rating_quant'] = quantize_rating[2]
playstore.loc[playstore['Rating'] >4.5, 'Rating_quant'] = quantize_rating[3]

results = pd.DataFrame(columns=['Type', 'Rating', 'nr/nd', 'Count'])
playstore['combined'] = playstore['Type'] + " / " + playstore['Rating_quant'] + " / " + playstore['nr/nd_quant']

counts = playstore['combined'].value_counts()

for z in range(0,len(quantize_type)):
    for j in range(0,len(quantize_rating)):
        for i in range(0,len(quantize_nr_nd)):
            key = f"{quantize_type[z]} / {quantize_rating[j]} / {quantize_nr_nd[i]}"
            #print("Key: ", key) # per vedere come si chiama la chiave (debug)
            count = counts.get(key, 0)
#Aggiungiamo i risultati al dataframe
            results.loc[len(results)] = [quantize_type[z], quantize_rating[j], quantize_nr_nd[i], count]
# printiamo il valore massimo di count
print("Massimo: ", results['Count'].max())
# ordiniamo il dataframe results in base alla colonna count
results = results.sort_values(by='Count', ascending=False)
results.head(24) # tutte le possibili combinazioni sono 24

Da questa visualizzazione possiamo notare che
* lo store è dominato da applicazioni che hanno un nr/nd basso
* il numero di app a pagamento è molto più piccolo rispetto al numero di app free
* generalmente le recensioni positive sono più comuni di quelle negative

Proviamo a visualizzare queste informazioni con qualche grafico:

In [ ]:
# grafico a torta per vedere la distribuzione delle app free/paid:
playstore['Type'].value_counts().plot.pie(autopct='%1.1f%%', figsize=(6,6), labels=None)
plt.ylabel('')
plt.title('Distribuzione delle app a pagamento e gratis')
plt.legend(labels=playstore['Type'].value_counts().index, loc='upper right', bbox_to_anchor=(1.3, 1))
plt.show()


In [ ]:
percentages = (playstore['nr/nd_quant'].value_counts() / len(playstore)) * 100
labels = [f'{label} - {percentage:.1f}%' for label, percentage in zip(percentages.index, percentages)]

# Plot del grafico a torta
playstore['nr/nd_quant'].value_counts().plot.pie(autopct='', figsize=(6,6), labels=None)
plt.ylabel('')
plt.title('Distribuzione delle app in base a nr/nd')
plt.legend(labels=labels, loc='upper right', bbox_to_anchor=(1.3, 1))
# Mostra il grafico
plt.show()

In [ ]:
low_results = results.loc[results['nr/nd'] == 'LOW']
medium_results = results.loc[results['nr/nd'] == 'MEDIUM']
high_results = results.loc[results['nr/nd'] == 'HIGH']

In [ ]:
custom_rating_order = ['BAD', 'SUFFICIENT', 'GOOD', 'EXCELLENT']

# Crea una copia esplicita del DataFrame per evitare il SettingWithCopyWarning
low_results_copy = low_results.copy()
medium_results_copy = medium_results.copy()
high_results_copy = high_results.copy()

# Modifica il Rating nel DataFrame copiato
low_results_copy['Rating'] = pd.Categorical(low_results_copy['Rating'], categories=custom_rating_order, ordered=True)
low_results_copy = low_results_copy.sort_values('Rating')
low_results=low_results_copy


medium_results_copy['Rating'] = pd.Categorical(medium_results_copy['Rating'], categories=custom_rating_order, ordered=True)
medium_results_copy = medium_results_copy.sort_values('Rating')
medium_results=medium_results_copy

high_results_copy['Rating'] = pd.Categorical(high_results_copy['Rating'], categories=custom_rating_order, ordered=True)
high_results_copy = high_results_copy.sort_values('Rating')
high_results=high_results_copy

In [ ]:
import matplotlib.pyplot as plt
# Crea una nuova figura
plt.figure(figsize=(10, 16))

# Plot per LOW
plt.subplot(3, 1, 1)  # 3 righe, 1 colonna, subplot 1
for type in low_results['Type'].unique():
    subset = low_results[low_results['Type'] == type]
    plt.plot(subset['Rating'], subset['Count'], marker='o', label=type)
plt.title('Count vs Rating su app con nr/nd LOW')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.legend()

# Plot per MEDIUM
plt.subplot(3, 1, 2)  # 3 righe, 1 colonna, subplot 2
for type in medium_results['Type'].unique():
    subset = medium_results[medium_results['Type'] == type]
    plt.plot(subset['Rating'], subset['Count'], marker='o', label=type)
plt.title('Count vs Rating su app con nr/nd MEDIUM')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.legend()

# Plot per HIGH
plt.subplot(3, 1, 3)  # 3 righe, 1 colonna, subplot 3
for type in high_results['Type'].unique():
    subset = high_results[high_results['Type'] == type]
    plt.plot(subset['Rating'], subset['Count'], marker='o', label=type)
plt.title('Count vs Rating su app con nr/nd HIGH')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.legend()

# Imposta lo spazio tra i grafici
plt.tight_layout()

# Mostra il grafico
plt.show()


Da questi ultimi grafici notiamo che, per quanto riguarda le app con nr/nd basso e medio la valutazione è sempre per lo più buona, mentre, se esaminiamo quelle con nr/nd alto, la valutazione maggioritaria è eccellente, sia per le app free che quelle a pagamento.
<br>
Un'altra cosa importante da notare è che la spezzata delle app free sta sempre sopra o al limite coincide con quella delle app a pagamento per quanto riguarda lo slicing su LOW e HIGH, mentre nel caso delle app con nr/nd MEDIUM questa tendenza si inverte sulle app valutate GOOD, ciò significa che in questo caso, ci sono più app considerate buone a pagamento che gratis (mentre però, come osserviamo, ci sono più app eccellenti gratis che a pagamento) 

## Test Statistico
Dopo aver condotto l'analisi, abbiamo identificato la categoria "game" come quella con il valore più elevato di nr/nd.
Presumiamo che, in questa categoria, saranno le applicazioni a pagamento ad avere l'nr-nd più alto. Questa supposizione si basa sul ragionamento che gli utenti che pagano per un'applicazione sono probabilmente più inclini a lasciare recensioni e quindi dare una valutazione, positiva o negativa che sia, all'applicazione. Questo perché l'atto di acquisto potrebbe indicare un maggiore coinvolgimento o aspettative più elevate da parte dell'utente rispetto alle app gratuite.

In [ ]:
#Creiamo un dataframe per la categoria game:
game = playstore[playstore['Category'] == 'GAME']
#Contiamo quante app sono a pagamento e quante sono gratuite:
game['Type'].value_counts()


Scegliamo un livello di significatività di 0.05 e consideriamo le seguenti ipotesi:
* ipotesi nulla H0 = le medie di nr/nd delle app free e di quelle paid **non** sono statisticamente diverse
* ipotesti alternativa HA = le medie sono diverse

In [ ]:
#Testiamo se le due distribuzioni sono significativamente diverse:
t_stat, p_value=stats.ttest_ind(game[game['Type'] == 'Free']['nr/nd'], game[game['Type'] == 'Paid']['nr/nd'])
print("p value",p_value)


Il p-value è molto basso, e, siccome p-value << livello di significatività, possiamo concludere che le due distribuzioni sono significativamente diverse (rigettando l'ipotesi nulla in favore di quella alternativa).

In [ ]:
#Infine, visualizziamo un boxplot per confrontare le due distribuzioni:
game.boxplot(column='nr/nd', by='Type', figsize=(10,10))
plt.ylabel('nr/nd')
plt.title('')
plt.suptitle('')
plt.show()

In [ ]:
#Ora vediamo tra quelle gratis e a pagamento chi ha la media nr/nd più alta:
print(game.groupby('Type')['nr/nd'].mean())


### Testiamo adesso la differenza tra reviews delle app free e di quelle paid

Scegliamo un livello di significatività di 0.05 e consideriamo le seguenti ipotesi:
* ipotesi nulla H0 = le medie di reviews delle app free e di quelle paid **non** sono statisticamente diverse
* ipotesti alternativa HA = le medie sono diverse

In [ ]:
free_reviews = game[game['Type'] == 'Free']['Reviews']
paid_reviews = game[game['Type'] == 'Paid']['Reviews']

# Testiamo se le due distribuzioni delle recensioni sono significativamente diverse
t_stat_reviews, p_value_reviews = stats.ttest_ind(free_reviews, paid_reviews)
print("T-Statistic (Reviews):", t_stat_reviews)
print("P-Value (Reviews):", p_value_reviews)

# Verifichiamo se esiste una differenza significativa nel numero di recensioni tra applicazioni gratuite e a pagamento
if p_value_reviews < 0.05:
    print("La differenza nel numero di recensioni tra applicazioni gratuite e a pagamento è statisticamente significativa.")
else:
    print("Non ci sono evidenze sufficienti per affermare che la differenza nel numero di recensioni tra applicazioni gratuite e a pagamento sia statisticamente significativa.")

Attraverso questi due test statistici abbiamo dunque scoperto che, per quanto riguarda la categoria GAME, c'è una differenza statisticamente significativa in termini di numero di recensioni/numero di installazioni tra app a pagamento e app gratuite, ma non c'è una differenza significativa in termini solo di recensioni.
Questo potrebbe indicare che le persone che pagano per un'applicazione sono più propense a dare recensioni rispetto a coloro che scaricano app gratuite, ma il numero totale di download delle app a pagamento è inferiore rispetto alle app gratuite

In [ ]:
filtered_apps = playstore[(playstore['Category'] == 'GAME') & (playstore['Type'] == 'Free')]
average_reviews = filtered_apps['Reviews'].mean()
print("Media delle recensioni per le applicazioni di gioco gratuite:", average_reviews)

filtered_apps = playstore[(playstore['Category'] == 'GAME') & (playstore['Type'] == 'Paid')]
average_reviews = filtered_apps['Reviews'].mean()
print("Media delle recensioni per le applicazioni di gioco a pagamento:", average_reviews)


## Metodo predittivo
Utilizziamo una rappresentazione grafica per verificare se le due features (nr/nd e rating) siano correlate, e se sia quindi possibile utilizzare una di queste features per predire l'altra.

In [ ]:
plt.scatter(playstore['nr/nd'], playstore['Rating'])
plt.title('scores')
plt.xlabel('nr/nd')
plt.ylabel('Rating')

Verifichiamo la correlazione attraverso il coefficiente di correlazione di Pearson (r).

In [ ]:
correlation, p_value = stats.pearsonr(playstore['nr/nd'],playstore['Rating'])
print("Coefficiente di Pearson: ", correlation)


### Terza ipotesi smentita
In fase preliminare avevamo pensato che, al crescere di una metrica, sarebbe diminuita l'altra, mentre, da quanto si evince dal calcolo del coefficiente di Pearson, c'è una correlazione positiva (anche se non particolarmente significativa) e non negativa tra le due variabili.

In [ ]:
# Calcoliamo una rappresentazione a griglia della densità dei punti
x = playstore['nr/nd']
y = playstore['Rating']
heatmap, xedges, yedges = np.histogram2d(x, y, bins=50)
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

# Plottiamo il grafico a dispersione in falsi colori
plt.imshow(heatmap.T, extent=extent, origin='lower', cmap='viridis')
plt.colorbar(label='Density')
plt.title('Density Plot of Ratings')
plt.xlabel('nr/nd')
plt.ylabel('Rating')
plt.show()

Notiamo anche in questo grafico una predominanza di rating alti

In [ ]:
#diamo un'occhiata alla distribuzione della colonna rating
playstore['Rating'].describe()

In [ ]:
moda = playstore['Rating'].mode()

moda_valore = moda[0] if not moda.empty else None

print("La moda è:", moda_valore)


### Distribuzione del Rating
Effettivamente vediamo che la media è 4.17 e che c'è una deviazione standard di 0.4, quindi intuiamo che la grande maggioranza delle app che stiamo analizzando hanno un rating che sta sulla fascia del 4/4.5. Calcolando la moda vediamo inoltre che il valore più frequente per il rating è 4.3.<br>
Tutte queste informazioni potrebbero suggerire che, se qualcuno lascia una recensione, in generale è per esprimere una opinione positiva (come vediamo, la distribuzione dei rating è sbilanciata sulla fascia medio-alta)

In [ ]:
plt.hist(playstore['Rating'], bins=20)
plt.xlabel('Rating')
plt.ylabel('Frequenza')
plt.title('Distribuzione di Rating' )
plt.show()

In [ ]:
#Plottiamo in falsi color con un hexbin plot:
plt.hexbin(playstore['nr/nd'], playstore['Rating'], gridsize=50, cmap='YlGnBu')
plt.xlabel('nr/nd')
plt.ylabel('Rating')
plt.title('Rating vs nr/nd')
plt.colorbar(label='count')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
#codice qui 
# Definiamo le caratteristiche da usare
X = playstore[['nr/nd']] #nr/nd sarà il nostro predittore
y = playstore['Rating'] #Rating sarà la variabile da predire (target)

# Dividiamo il dataset in training set e validation set usando la proporzione 70/30
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
# Istanziamo l'oggetto della classe LinearRegression
regressor = LinearRegression()

# Alleniamo il modello passando il training set
regressor.fit(X_train, y_train)

# Visualizziamo i coefficienti imparati
print("Intercept:", regressor.intercept_)
print("Coefficiente nr/nd", regressor.coef_[0]) 


In [ ]:
y_hat =regressor.predict(X_valid)

plt.scatter(X_valid,y_hat)
plt.scatter(X_valid,y_valid,c='r',alpha=0.2)

### Usiamo le tre stime per capire quanto precisa è la funzione imparata


In [ ]:
import sklearn.metrics as metrics 
y_pred = regressor.predict(X_valid)
print ('MAE:', metrics.mean_absolute_error(y_valid, y_pred))
print ('MSE:', metrics.mean_squared_error(y_valid, y_pred))
print ('RMSE:', np.sqrt(metrics.mean_squared_error(y_valid, y_pred)))

### Determiniamo il modello nullo e confrontiamo le metriche appena ottenute

In [ ]:
def MSE (y_pred,y_true):
    #errore quadratico medio
    mse = np.mean((y_pred - y_true) ** 2)
    return mse 

def MAE (y_pred,y_true):
    #scarto medio assoluto
    mae = np.mean(np.abs(y_pred - y_true))
    return mae


def RMSE (y_pred,y_true):
    #radice quadrata dell'errore quadratico medio
    rmse = np.sqrt(np.mean((y_pred - y_true) ** 2))
    return rmse


# calcoliamo il nr/nd medio
mean_nr_nd = playstore['nr/nd'].mean()
# creiamo la nuova x con lo stesso numero di campioni del validation set originale, e con tutti i valori uguali a mean_nr_nd appena calcolato
null_model = [mean_nr_nd] * y_valid.shape[0]

null_mae = MAE(null_model, y_valid)
null_mse = MSE(null_model, y_valid)
null_rmse = RMSE(null_model, y_valid)

#stampate gli errori
print("Scarto medio assoluto (MAE):", null_mae)
print("Errore quadratico medio (MSE):", null_mse)
print("Radice dell'errore quadratico medio (RMSE):", null_rmse)

### Considerazioni
Possiamo dire quindi che il modello sta imparando, ma non riporta risultati troppo sorprendenti, è sicuramente però migliore del modello nullo, in quanto, sia lo scarto medio assoluto che l'errore quadratico medio che la radice dell'errore quadratico medio sono minori rispetto a quelli ottenuti con il modello nullo.

In [ ]:
playstore.info() #visualizziamo la struttura del dataset prima di effettuare clustering

## Clustering
Innanzitutto calcoliamo e plottiamo il coefficiente di silhouette per ogni k da 2 a 10

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
X = playstore[['Rating', 'nr/nd', 'Installs', 'Reviews']]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Lista dei numeri di cluster da testare
num_clusters = range(2, 11)  # Possiamo testare da 2 a 10 cluster

# Lista per memorizzare i coefficienti di silhouette corrispondenti a ciascun numero di cluster
silhouette_scores = []

# Calcola il coefficiente di silhouette per ciascun numero di cluster
for n_clusters in num_clusters:
    model = KMeans(n_clusters=n_clusters, random_state=42)
    model.fit(X_scaled)
    silhouette_avg = silhouette_score(X_scaled, model.labels_)
    silhouette_scores.append(silhouette_avg)

# Traccia il grafico
plt.plot(num_clusters, silhouette_scores, marker='o')
plt.xlabel('Numero di Cluster')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Numero di Cluster')
plt.xticks(num_clusters)
plt.grid(True)
plt.show()


Scegliamo k=5 perché sembra essere quello con silhouette migliore (oltre al 2 che è abbastanza scontato)

In [ ]:
#Facciamo un esempio di clustering con i dati a disposizione:

#Addestriamo il modello di clustering:
model = KMeans(n_clusters=5, random_state=42)
model.fit(X_scaled)

#Aggiungiamo le predizioni al dataset:
playstore['Cluster'] = model.predict(X_scaled)

# Visualizziamo il numero di app per cluster
playstore['Cluster'].value_counts()

# Visualizziamo le statistiche per ogni cluster
playstore.groupby('Cluster').describe()

In [ ]:
# Visualizziamo le app di ogni cluster
for cluster_num in range(model.n_clusters):
    print(f"Cluster {cluster_num}:\n")
    print(playstore[playstore['Cluster'] == cluster_num].head())
    print("\n")

In [ ]:
playstore['Cluster']

In [ ]:
playstore[(playstore['Cluster'] == 3) & (playstore['Rating_quant'] == 'BAD')]

### Considerazioni sul clustering 

In [ ]:
playstore.describe()